# Source Code For SER

This code implment MLP, SVM, Naive Bayes, and Voting-Based Ensemble Classiefer to rocognize human emotions using the dataset imported from the RAVDESS.

# Required Imports

In [218]:
# Code was based on DataFlair Team Project
# Code was based on DataCamp Tutorials
# Dataset was taken from (RAVDESS)

import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Defining The Function For Extracting Features From The Sound File (mfcc, chroma, mel) 

In [219]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

# Making A Dictionary Of The Emotions

In [220]:
#Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust'] #You specify the emotions you want to observe 

# Loading The Data & Extract The Features From Each File

In [221]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("C:\\Users\hashi\Project\speech-emotion-recognition-ravdess-data\Actor_*\\*.wav"): #Set the path of the dataset
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

# Splitting The Dataset (Training & Testing Sets)

In [222]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

# Shape Of The Dataset

In [223]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


# Number Of Features

In [224]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


# (1) Initialize MLP Classifier

In [225]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

# Train The Model (Fit)

In [226]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_iter=500, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

# Predict The Value Of Test Set

In [227]:
#Predict for the test set
y_pred=model.predict(x_test)

# Accuracy Score

In [228]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 67.71%


# (2) Initialize SVM

In [229]:
#Import svm model
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

# Train The Model (Fit)

In [230]:
#Train the model using the training sets
clf.fit(x_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

# Predict The Value Of Test Set

In [231]:
#Predict the response for test dataset
y_pred = clf.predict(x_test)

# Accuracy Score

In [232]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6302083333333334


# (3) Initialize Naive Bayes Classification

In [233]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
gnb = GaussianNB()


# Train The Model (Fit)

In [234]:
#Train the model using the training sets
gnb.fit(x_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

# Predict The Value Of The Test Set

In [235]:
#Predict the response for test dataset
y_pred = gnb.predict(x_test)

# Accuracy Score

In [236]:
#Import scikit-learn metrics module for accuracy calculation
import matplotlib.pyplot as plt  # doctest: +SKIP
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.4895833333333333


# Voting Ensemble Classification

In [237]:
# Voting Ensemble for Classification
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn import model_selection

seed = 7

kfold = model_selection.KFold(n_splits=10, random_state=seed)
# create the sub models
estimators = []
svm_vote = svm.SVC(kernel='linear')
estimators.append(('svm', model1))
mlp_vote = MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)
estimators.append(('mlp', model2))
nbs_vote = gnb = GaussianNB()
estimators.append(('nbs', model3))
# create the ensemble model
ensemble = VotingClassifier(estimators)
results = model_selection.cross_val_score(ensemble, x_train, y_train, cv=kfold)

# Accuracy Score of Voting-Based (mean)

In [238]:
 print("Accuracy Score (mean): ", results.mean())

Accuracy Score (mean):  0.6825166364186328
